# Knapsack Optimization with KOPPU

This notebook explores the Knapsack problem and the effect of the penalty parameter.

In [ ]:
from pykoppu.problems.logistics import Knapsack
from pykoppu import Process
import matplotlib.pyplot as plt

# 1. Define Items and Capacity
items = [
    {'value': 10, 'weight': 5},
    {'value': 5, 'weight': 3},
    {'value': 15, 'weight': 8},
    {'value': 7, 'weight': 4},
    {'value': 6, 'weight': 2}
]
capacity = 10

# 2. Solve with different penalties
penalties = [0.1, 1.0, 10.0]
results = []

for p in penalties:
    problem = Knapsack(items, capacity, penalty=p)
    process = Process(problem, backend="brian2")
    res = process.run()
    results.append((p, res['state']))
    print(f"Penalty {p}: Done")

In [ ]:
# 3. Analyze Results
for p, state in results:
    # Convert state to binary selection
    selection = [1 if v > -0.06 else 0 for v in state]
    
    total_weight = sum(item['weight'] for i, item in enumerate(items) if selection[i])
    total_value = sum(item['value'] for i, item in enumerate(items) if selection[i])
    
    print(f"Penalty: {p}")
    print(f"Selection: {selection}")
    print(f"Total Weight: {total_weight} (Capacity: {capacity})")
    print(f"Total Value: {total_value}")
    print("Valid:", total_weight <= capacity)
    print("-" * 20)